### Welcome to Nikolaj

This project as a whole is the work of Anand Chauha, Manya Sharma, and Vasu Jain. But this document will be created and worked on only by Anand Chauhan. I intend to use this notebook as a document to my working process for this hackathon.

In [23]:
# importing all the libraries i will need
import numpy as np
import pandas as pd
import sklearn as sl
import folium as fl
import csv
import datetime as dt
import matplotlib.pyplot as plt

In [24]:
# loading in the dataset for the sample
# updated to load the filtered dataset
try:
    dataset = pd.read_csv('datasets/crimedata2016.csv')
except:
    dataset = pd.read_csv('datasets/Chicago_Crimes_2012_to_2017.csv')
dataset.head()

,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,FBI Code,X Coordinate,Y Coordinate,Year,Location
0,05/03/2016 11:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,08B,1154907.0,1893681.0,2016,"(41.864073157, -87.706818608)"
1,05/03/2016 09:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,08B,1183066.0,1864330.0,2016,"(41.782921527, -87.60436317)"
2,05/03/2016 11:31:00 PM,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,24,1140789.0,1904819.0,2016,"(41.894908283, -87.758371958)"
3,05/03/2016 10:10:00 PM,BATTERY,SIMPLE,SIDEWALK,False,False,1532,08B,1143223.0,1901475.0,2016,"(41.885686845, -87.749515983)"
4,05/03/2016 10:00:00 PM,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,06,1139890.0,1901675.0,2016,"(41.886297242, -87.761750709)"


In [30]:
# dropping all information not needed by our program.
try:
    dataset = dataset.drop(['Unnamed: 0', 'ID', 'Case Number', 'Block', 'IUCR', 'Updated On', 'Latitude', 
            'Longitude', 'District', 'Ward', 'Community Area'], axis=1)
except:
    pass

# limiting our data to just a year.
dataset_train = dataset[dataset['Year'] == 2016]
dataset_test = dataset[dataset['Year'] == 2015]
dataset

,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,FBI Code,X Coordinate,Y Coordinate,Year,Location
0,05/03/2016 11:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,08B,1154907.0,1893681.0,2016,"(41.864073157, -87.706818608)"
1,05/03/2016 09:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,08B,1183066.0,1864330.0,2016,"(41.782921527, -87.60436317)"
2,05/03/2016 11:31:00 PM,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,24,1140789.0,1904819.0,2016,"(41.894908283, -87.758371958)"
3,05/03/2016 10:10:00 PM,BATTERY,SIMPLE,SIDEWALK,False,False,1532,08B,1143223.0,1901475.0,2016,"(41.885686845, -87.749515983)"
4,05/03/2016 10:00:00 PM,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,06,1139890.0,1901675.0,2016,"(41.886297242, -87.761750709)"
...,...,...,...,...,...,...,...,...,...,...,...,...
265457,05/03/2016 11:33:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1034,08B,1159105.0,1888300.0,2016,"(41.849222028, -87.69155551)"
265458,05/03/2016 11:30:00 PM,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,True,True,731,14,1175230.0,1856183.0,2016,"(41.760743949, -87.63333531)"
265459,05/03/2016 12:15:00 AM,BATTERY,AGGRAVATED: HANDGUN,SIDEWALK,False,False,825,04B,1161027.0,1862810.0,2016,"(41.779234743, -87.685207125)"
265460,05/03/2016 09:07:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,False,True,423,08B,1197261.0,1850727.0,2016,"(41.745251975, -87.552773464)"


In [34]:
# script to save all the data, for import and analysis later
# NOT NEEDED TO BE RUN ANYMORE

f = open('datasets/crimedata2016.csv', 'w')
writer = csv.writer(f)
writer.writerow(['Date', 'Primary Type', 'Description', 'Location Description', 'Arrest',
                'Domestic', 'Beat', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year',
                'Location', 'Latitude', 'Longitude'])
for i in range(len(dataset_train)):
    writer.writerow(dataset_train.iloc[i])
f.close()
# f = open('datasets/crimedata2017.csv', 'w')
# writer = csv.writer(f)
# writer.writerow(['Date', 'Primary Type', 'Description', 'Location Description', 'Arrest',
#                 'Domestic', 'Beat', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year',
#                 'Location'])
# for i in range(len(dataset_test)):
#     writer.writerow(dataset_test.iloc[i])
# f.close()

In [5]:
basicCrime = list(set(dataset_train['Primary Type']))

In [42]:
# convert location into lat, long lists.
locset = []
loclat = []
loclong = []
dataset_train = dataset_train.dropna()
for i in dataset_train['Location']:
    locset.append([float(i[0]), float(i[1])])
    loclat.append(float(i[0]))
    loclong.append(float(i[1]))

dataset_train['Location'] = locset
dataset_train['Latitude'] = loclat
dataset_train['Longitude'] = loclong
dataset_train.head()

,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,FBI Code,X Coordinate,Y Coordinate,Year,Location,Latitude,Longitude
0,05/03/2016 11:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,08B,1154907.0,1893681.0,2016,"[41.864073157, -87.706818608]",41.864073,-87.706819
1,05/03/2016 09:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,08B,1183066.0,1864330.0,2016,"[41.782921527, -87.60436317]",41.782922,-87.604363
2,05/03/2016 11:31:00 PM,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,24,1140789.0,1904819.0,2016,"[41.894908283, -87.758371958]",41.894908,-87.758372
3,05/03/2016 10:10:00 PM,BATTERY,SIMPLE,SIDEWALK,False,False,1532,08B,1143223.0,1901475.0,2016,"[41.885686845, -87.749515983]",41.885687,-87.749516
4,05/03/2016 10:00:00 PM,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,06,1139890.0,1901675.0,2016,"[41.886297242, -87.761750709]",41.886297,-87.761751
